In [1]:
import cv2
import requests
import json
from pyzbar import pyzbar

#launch and run the scanner
def read_barcodes(frame, barcode_info):
    barcodes = pyzbar.decode(frame)
    for barcode in barcodes:
        x, y , w, h = barcode.rect
        barcode_info = barcode.data.decode('utf-8')
        cv2.rectangle(frame, (x, y),(x+w, y+h), (0, 255, 0), 2)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, barcode_info, (x + 6, y - 6), font, 2.0, (255, 255, 255), 1)
    return frame, barcode_info

def call_scanner():
    camera = cv2.VideoCapture(0,cv2.CAP_DSHOW)
    ret, frame = camera.read()
    barcode_info = ""
    while ret:
        ret, frame = camera.read()
        frame, barcode_info = read_barcodes(frame, barcode_info)
        cv2.imshow('Barcode/QR code reader', frame)
        if cv2.waitKey(1) and len(barcode_info) == 13:
            break
    camera.release()
    cv2.destroyAllWindows()
    return barcode_info


In [2]:
print("Scan barcode of the book")
isbn = call_scanner()
good_isbn = input("Is this the right barcode: {} (yes or no)? ".format(isbn))
if good_isbn.lower() == "yes":
    url = "http://127.0.0.1:5000/add_book/"
    payload = json.dumps({
    "isbn": isbn
    })
    headers = {
    'Content-Type': 'application/json'
    }
    response = requests.post(url, headers=headers, data=payload)
    response.raise_for_status()
    print(response.text)
else:
    print("Try again")   

Scan barcode of the book
Try again
